In [43]:
import deepchem as dc
import pandas as pd


# Studying Ways of visualization of 

In [37]:
# Load BACE regression dataset
# featurizer =
bace_tasks, bace_datasets, transformers = dc.molnet.load_bace_regression(featurizer='ECFP')

In [45]:
# Create a DataFrame 
smiles = bace_datasets[0].ids
targets = bace_datasets[0].y
fingerprints = bace_datasets[0].X

# Create 2 Dataframes: 1 with fingerprints and targets, and 1 with smiles and targets
df = pd.DataFrame(fingerprints)
df['target'] = targets

df_smiles = pd.DataFrame(smiles, columns=['smiles'])
df_smiles['target'] = targets


In [63]:
# Compute Tanimoto similarity matrix
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np

# Convert fingerprints to RDKit format
ms = [Chem.MolFromSmiles(x) for x in smiles]
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=1024) for x in ms]

# Create a similarity list
query, target, similarity = [], [], []
for n in range(len(fingerprints)-1):
    for m in range(n+1, len(fingerprints)):
        query.append(smiles[n])
        target.append(smiles[m])
        similarity.append(DataStructs.TanimotoSimilarity(fingerprints[n], fingerprints[m]))



In [69]:
# Df
df_similarity = pd.DataFrame({'query': query, 'target': target, 'similarity': similarity})
# Build a similarity matrix
similarity_matrix = np.zeros((len(smiles), len(smiles)))
for i in range(len(query)):
    n = df_smiles.index[df_smiles['smiles'] == query[i]].tolist()[0]
    m = df_smiles.index[df_smiles['smiles'] == target[i]].tolist()[0]
    similarity_matrix[n, m] = similarity[i]
    similarity_matrix[m, n] = similarity[i]
    
similarity_matrix